# Apple Logo Detection: Dataset + Model

This notebook builds a dataset from your transparent Apple logos in `true_apple`,
generates many varied training images, and trains a detector.


## 0. Install dependencies

We use `ultralytics` (YOLOv8) for detection. If you are offline,
install from a wheel in advance.


In [ ]:
# !pip install ultralytics tqdm pyyaml pillow numpy


## 1. Project structure

Expected structure:
````
apple_case/
  true_apple/          # transparent Apple logos (png/svg/etc)
  data/
    labeled/
      images/
      labels/          # YOLO txt labels
    splits/
      train/
      val/
  runs/
  data_and_mpdel.ipynb
````


In [5]:
from pathlib import Path

root = Path.cwd()
data_dir = root / 'data'
(data_dir / 'labeled' / 'images').mkdir(parents=True, exist_ok=True)
(data_dir / 'labeled' / 'labels').mkdir(parents=True, exist_ok=True)
(data_dir / 'splits' / 'train').mkdir(parents=True, exist_ok=True)
(data_dir / 'splits' / 'val').mkdir(parents=True, exist_ok=True)


## 2. Generate synthetic dataset from real logos

This cell composites your transparent logos onto varied backgrounds,
producing labels automatically. It handles common raster formats and
skips SVG if `cairosvg` is not installed.


In [2]:
# import io
# import random
# import math
# from pathlib import Path
# import numpy as np
# from PIL import Image, ImageDraw, ImageFilter

# random.seed(42)
# np.random.seed(42)

# src_dir = root / 'true_apple'
# out_images = data_dir / 'labeled' / 'images'
# out_labels = data_dir / 'labeled' / 'labels'

# raster_exts = {'.png', '.jpg', '.jpeg', '.bmp', '.webp', '.tif', '.tiff'}
# svg_exts = {'.svg'}

# def load_logo(path):
#     ext = path.suffix.lower()
#     if ext in raster_exts:
#         img = Image.open(path).convert('RGBA')
#     elif ext in svg_exts:
#         try:
#             import cairosvg  # optional
#         except Exception:
#             return None
#         png_bytes = cairosvg.svg2png(url=str(path))
#         img = Image.open(io.BytesIO(png_bytes)).convert('RGBA')
#     else:
#         return None

#     alpha = img.split()[3]
#     bbox = alpha.getbbox()
#     if bbox is None:
#         return None
#     return img.crop(bbox)

# def random_background(w, h):
#     mode = random.choice(['solid', 'gradient', 'noise', 'stripes'])
#     if mode == 'solid':
#         color = tuple(random.randint(0, 255) for _ in range(3))
#         return Image.new('RGB', (w, h), color)
#     if mode == 'gradient':
#         c1 = np.array([random.randint(0, 255) for _ in range(3)], dtype=np.float32)
#         c2 = np.array([random.randint(0, 255) for _ in range(3)], dtype=np.float32)
#         t = np.linspace(0, 1, h).reshape(h, 1, 1)
#         grad = (c1 * (1 - t) + c2 * t).astype(np.uint8)
#         grad = np.repeat(grad, w, axis=1)
#         return Image.fromarray(grad, mode='RGB')
#     if mode == 'noise':
#         noise = np.random.randint(0, 256, (h, w, 3), dtype=np.uint8)
#         return Image.fromarray(noise, mode='RGB')
#     # stripes
#     base = Image.new('RGB', (w, h), tuple(random.randint(0, 255) for _ in range(3)))
#     d = ImageDraw.Draw(base)
#     for y in range(0, h, random.randint(20, 60)):
#         stripe_color = tuple(random.randint(0, 255) for _ in range(3))
#         d.rectangle([0, y, w, y + random.randint(8, 20)], fill=stripe_color)
#     return base

# def colorize_logo(logo):
#     if random.random() < 0.7:
#         color = tuple(random.randint(0, 255) for _ in range(3))
#         colored = Image.new('RGB', logo.size, color)
#         colored.putalpha(logo.split()[3])
#         return colored
#     return logo

# logos = []
# for p in sorted(src_dir.iterdir()):
#     if p.is_file():
#         img = load_logo(p)
#         if img is not None:
#             logos.append(img)

# if not logos:
#     raise RuntimeError('No usable logos found in true_apple')

# num_images = 1200
# img_sizes = [(640, 640), (512, 512), (800, 600), (600, 800), (1024, 768)]

# for i in range(num_images):
#     bg_w, bg_h = random.choice(img_sizes)
#     bg = random_background(bg_w, bg_h)

#     logo = random.choice(logos)
#     logo = colorize_logo(logo)

#     target = random.uniform(0.2, 0.6) * min(bg_w, bg_h)
#     scale = target / max(logo.size)
#     new_w = max(1, int(logo.width * scale))
#     new_h = max(1, int(logo.height * scale))
#     logo = logo.resize((new_w, new_h), Image.BICUBIC)

#     angle = random.uniform(-25, 25)
#     logo = logo.rotate(angle, expand=True, resample=Image.BICUBIC)

#     if logo.width >= bg_w or logo.height >= bg_h:
#         shrink = min(bg_w / (logo.width + 1), bg_h / (logo.height + 1)) * 0.9
#         logo = logo.resize((max(1, int(logo.width * shrink)), max(1, int(logo.height * shrink))), Image.BICUBIC)

#     x = random.randint(0, bg_w - logo.width)
#     y = random.randint(0, bg_h - logo.height)
#     bg.paste(logo, (x, y), logo)

#     # bounding box from alpha mask
#     alpha = logo.split()[3]
#     bbox = alpha.getbbox()
#     if bbox is None:
#         continue
#     x_min, y_min, x_max, y_max = bbox
#     x_min += x
#     x_max += x
#     y_min += y
#     y_max += y

#     x_c = (x_min + x_max) / 2.0 / bg_w
#     y_c = (y_min + y_max) / 2.0 / bg_h
#     w = (x_max - x_min) / bg_w
#     h = (y_max - y_min) / bg_h

#     if random.random() < 0.25:
#         bg = bg.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.3, 1.2)))

#     img_name = f'logo_{i:05d}.jpg'
#     label_name = f'logo_{i:05d}.txt'
#     bg.save(out_images / img_name, quality=95)
#     with open(out_labels / label_name, 'w', encoding='utf-8') as f:
#         f.write(f'0 {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\\n')

# print('Generated images:', num_images)


Generated images: 1200


## 3. Train/val split

Create a reproducible split (e.g., 80/20).


In [7]:
# import random
# import shutil
# from pathlib import Path

# random.seed(42)

# images_dir = data_dir / 'labeled' / 'images'
# labels_dir = data_dir / 'labeled' / 'labels'

# train_dir = data_dir / 'splits' / 'train'
# val_dir = data_dir / 'splits' / 'val'

# for p in [train_dir / 'images', train_dir / 'labels', val_dir / 'images', val_dir / 'labels']:
#     p.mkdir(parents=True, exist_ok=True)

# images = sorted(images_dir.glob('*'))
# random.shuffle(images)

# split_idx = int(0.8 * len(images))
# train_images = images[:split_idx]
# val_images = images[split_idx:]

# def copy_pair(img_path, split_dir):
#     label_path = labels_dir / (img_path.stem + '.txt')
#     if not label_path.exists():
#         return
#     shutil.copy2(img_path, split_dir / 'images' / img_path.name)
#     shutil.copy2(label_path, split_dir / 'labels' / label_path.name)

# for img in train_images:
#     copy_pair(img, train_dir)
# for img in val_images:
#     copy_pair(img, val_dir)

# print('Train images:', len(train_images))
# print('Val images:', len(val_images))


## 4. YOLO dataset config

Create `data/apple.yaml` for training.


In [8]:
import yaml

data_yaml = {
    'path': str((data_dir / 'splits').resolve()),
    'train': 'train/images',
    'val': 'val/images',
    'names': ['apple_logo']
}

with open(data_dir / 'apple.yaml', 'w', encoding='utf-8') as f:
    yaml.safe_dump(data_yaml, f, allow_unicode=False)

print(data_yaml)


{'path': 'C:\\Users\\Danila\\apple_case\\data\\splits', 'train': 'train/images', 'val': 'val/images', 'names': ['apple_logo']}


## 5. Model training (YOLOv8)

Base model: `yolov8n.pt` (small, fast).
You can switch to `yolov8s.pt` for higher accuracy.


In [4]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [9]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(
    data=str(data_dir / "apple.yaml"),
    epochs=50,
    imgsz=640,
    batch=16,
    device=0,              # <-- GPU
    project="runs",
    name="apple_logo",
)


Ultralytics 8.3.250  Python-3.10.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060 SUPER, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\Danila\apple_case\data\apple.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=apple_logo2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100,

C:\Users\Danila\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-09 13:15:13,865	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2026-01-09 13:15:14,421	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000020A07C72F80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

## 6. Validation and metrics

Compute mAP, precision, recall on the validation set.


In [12]:
# If model is not in memory, load the trained weights:
# from ultralytics import YOLO
# model = YOLO('runs/apple_logo/weights/best.pt')

metrics = model.val(data=str(data_dir / 'apple.yaml'))
print(metrics.results_dict)

# Common metrics to report:
# - metrics/precision(B)
# - metrics/recall(B)
# - metrics/mAP50(B)
# - metrics/mAP50-95(B)


Ultralytics 8.3.250  Python-3.10.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060 SUPER, 8192MiB)
val: Fast image access  (ping: 0.10.0 ms, read: 145.4188.8 MB/s, size: 109.3 KB)
val: Scanning C:\Users\Danila\apple_case\data\splits\val\labels.cache... 438 images, 0 backgrounds, 240 corrupt: 100% ━━━━━━━━━━━━ 438/438  0.0s
val: C:\Users\Danila\apple_case\data\splits\val\images\logo_00001.jpg: ignoring corrupt image/label: could not convert string to float: '0.326172\\n'
val: C:\Users\Danila\apple_case\data\splits\val\images\logo_00006.jpg: ignoring corrupt image/label: could not convert string to float: '0.574219\\n'
val: C:\Users\Danila\apple_case\data\splits\val\images\logo_00013.jpg: ignoring corrupt image/label: could not convert string to float: '0.313750\\n'
val: C:\Users\Danila\apple_case\data\splits\val\images\logo_00014.jpg: ignoring corrupt image/label: could not convert string to float: '0.305000\\n'
val: C:\Users\Danila\apple_case\data\splits\val\images\logo_00023.jpg: i

## 7. Inference on your images

Check the model on your own files.


In [11]:
results = model.predict(source='data/labeled/images', save=True, conf=0.25)
print(results)



WARNING 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1200 C:\Users\Danila\apple_case\data\labeled\images\logo_00000.jpg: 640x640 1 apple_logo, 43.8ms
image 2/1200 C:\Users\Danila\apple_case\data\labeled\images\logo_00001.jpg: 640x640 1 apple_logo, 105.9ms
image 3/1200 C:\Users\Danila\apple_case\data\labeled\images\logo_00002.jpg: 480x640 1 apple_logo, 41.2ms
image 4/1200 C:\Users\Danila\apple_case\data\labeled\images\logo_00003.jpg: 480x640 1 apple_logo, 19.3ms
image 5/1200 C:\Users\Danila\apple_case\